### Ollama gpt-oss:20b 로드

python 3.12.3사용중

In [31]:
!nvidia-smi
!python -c "import torch,platform,sys; print('py',sys.version); print('cuda?',torch.cuda.is_available())"


Mon Sep 15 14:26:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:49:00.0 Off |                    0 |
| N/A   37C    P0             36W /  250W |   13041MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

#### ollama 서버에 설치

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%                                                            5.8%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#### ollama serve 시작

In [6]:
!pwd

/home/work/IROLIM/AIHumanities


In [8]:
import subprocess, os

In [5]:
# GPU 환경변수 설정 후 서버 실행
!export OLLAMA_USE_GPU=1
# 선택: 특정 GPU만 쓰려면 (예: 0번 GPU만)
!export CUDA_VISIBLE_DEVICES=0

In [13]:
!export OLLAMA_MODELS="../ollama"

In [21]:
# 주피터 프로세스 자체의 환경변수 수정
os.environ["OLLAMA_MODELS"] = "../../ollama"

# serve 띄울 때도 같은 os.environ 사용
subprocess.Popen(["ollama", "serve"], env=os.environ)


<Popen: returncode: None args: ['ollama', 'serve']>

time=2025-09-15T14:22:05.462+09:00 level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:../../ollama OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:f

In [22]:
!ollama list

[GIN] 2025/09/15 - 14:22:08 | 200 |      84.025µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/15 - 14:22:08 | 200 |     3.84879ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED   
gemma3:27b     a418f5838eaf    17 GB    8 days ago    
gpt-oss:20b    aa4295ac10c3    13 GB    8 days ago    


#### ollama server 종료

In [20]:

subprocess.run(["pkill", "-f", "ollama serve"], check=False)

CompletedProcess(args=['pkill', '-f', 'ollama serve'], returncode=0)

#### ollama pull llm

In [12]:
# 모델 풀
subprocess.run(["ollama", "pull", "gpt-oss:20b"], check=True, env=os.environ)

[GIN] 2025/09/15 - 13:45:00 | 200 |      79.129µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ time=2025-09-15T13:45:01.302+09:00 level=INFO source=download.go:177 msg="downloading b112e727c6f1 in 16 861 MB part(s)"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 729 KB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 1.4 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 2.8 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 5.9 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏  24 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   1% ▕                  ▏ 146 MB/ 13 GB           

[GIN] 2025/09/15 - 13:47:43 | 200 |         2m43s |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling b112e727c6f1: 100% ▕██████████████████▏  13 GB                         
pulling fa6710a93d78: 100% ▕██████████████████▏ 7.2 KB                         
pulling f60356777647: 100% ▕██████████████████▏  11 KB                         
pulling d8ba2f9a17b3: 100% ▕██████████████████▏   18 B                         
pulling 55c108d8e936: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gpt-oss:20b'], returncode=0)

In [14]:
# 모델 풀
subprocess.run(["ollama", "pull", "gemma3:27b"], check=True, env=os.environ)

[GIN] 2025/09/15 - 13:50:02 | 200 |      29.159µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ time=2025-09-15T13:50:03.630+09:00 level=INFO source=download.go:177 msg="downloading e796792eba26 in 18 1 GB part(s)"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 398 KB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 770 KB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 1.9 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 3.2 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 4.6 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏  69 MB/ 17 GB                  pulling manife

[GIN] 2025/09/15 - 13:52:50 | 200 |         2m48s |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest ⠸ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest ⠼ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a425

CompletedProcess(args=['ollama', 'pull', 'gemma3:27b'], returncode=0)

In [ ]:
# # ✅ Jupyter에서 Ollama 서버를 GPU 모드로 '비차단' 실행하기
# import os, subprocess, time, signal, requests

# # 1) 환경변수: GPU 사용
# env = os.environ.copy()
# env["OLLAMA_USE_GPU"] = "1"          # 서버 프로세스에 적용됨
# env["CUDA_VISIBLE_DEVICES"] = "0"    # 필요시 GPU 인덱스 지정

# # 2) 로그 파일
# LOG = "/tmp/ollama.log"
# logf = open(LOG, "ab", buffering=0)

# # 3) 기존 떠 있는 ollama serve가 있으면 정리(선택)
# try:
#     subprocess.run(["pkill", "-f", "ollama serve"], check=False)
#     time.sleep(1)
# except Exception:
#     pass

# # 4) 비차단 실행 (주피터에선 & 금지 → Popen으로 대체)
# #    preexec_fn=os.setpgrp 로 자식 프로세스 그룹을 분리(노트북 중지와 분리)
# proc = subprocess.Popen(
#     ["ollama", "serve"],
#     stdout=logf, stderr=subprocess.STDOUT,
#     env=env,
#     preexec_fn=os.setpgrp  # 리눅스/WSL2
# )

# # 5) 헬스체크
# time.sleep(2)
# try:
#     r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
#     print("✔️ Ollama up:", r.status_code, r.json())
# except Exception as e:
#     print("❌ 서버 확인 실패:", e)

# # 6) 최근 로그 몇 줄 보기
# try:
#     with open(LOG, "rb") as f:
#         tail = f.read().splitlines()[-20:]
#     print("\n--- /tmp/ollama.log (tail) ---")
#     for line in tail:
#         try:
#             print(line.decode("utf-8", "ignore"))
#         except:
#             print(line)
# except FileNotFoundError:
#     print("로그가 아직 생성되지 않았어요.")


✔️ Ollama up: 200 {'models': []}

--- /tmp/ollama.log (tail) ---
Couldn't find '/home/work/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIG9dTheXUiX9qqqba3u5lCbqP9qlGh+k1V4C4hqEf7Tw

time=2025-09-15T13:43:12.760+09:00 level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/work/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost h

#### ollama serve 특정 모델 위치에서 실행

### ollama - gpt-oss:20b

In [7]:
import os, subprocess

# 원하는 경로 지정
os.environ["OLLAMA_MODELS"] = "../ollama"

# 디렉토리가 없으면 먼저 생성
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# 모델 풀
subprocess.run(["ollama", "pull", "gpt-oss:20b"], check=True, env=os.environ)

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 1.5 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   1% ▕                  ▏  85 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   1% ▕                  ▏ 178 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   2% ▕                  ▏ 223 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   2% ▕                  ▏ 315 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   3% ▕                  ▏ 395 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   3% ▕                  ▏ 437 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   4% ▕         

CalledProcessError: Command '['ollama', 'pull', 'gpt-oss:20b']' returned non-zero exit status 1.

In [26]:
!pip install langchain langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPR

In [23]:
!pip install langchain-community

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPR

In [27]:
import requests
resp = requests.post(
    "http://127.0.0.1:11434/api/generate",
    json={"model": "gpt-oss:20b", "prompt": "GPU로 추론 중인지 확인하는 간단한 방법을 한 문장으로 말해줘(한국어).", "stream": False},
    timeout=120,
)
print(resp.json()["response"])


time=2025-09-15T14:24:32.460+09:00 level=INFO source=server.go:199 msg="model wants flash attention"
time=2025-09-15T14:24:32.460+09:00 level=INFO source=server.go:216 msg="enabling flash attention"
time=2025-09-15T14:24:32.460+09:00 level=WARN source=server.go:224 msg="kv cache type not supported by model" type=""
time=2025-09-15T14:24:32.460+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model ../../ollama/blobs/sha256-b112e727c6f18875636c56a779790a590d705aec9e1c0eb5a97d51fc2a778583 --port 45609"
time=2025-09-15T14:24:32.488+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-15T14:24:32.489+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:45609"
time=2025-09-15T14:24:33.530+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="845.4 GiB" free_swap="8.0 GiB"
time=2025-09-15T14:24:33.824+09:00 level=INFO source=memory.go:36 msg="new model 

[GIN] 2025/09/15 - 14:24:45 | 200 | 14.454059195s |       127.0.0.1 | POST     "/api/generate"
PyTorch에서는 `torch.cuda.is_available()`가 True이고 모델의 파라미터가 `device='cuda'`로 이동했는지(예: `model.to('cuda')`) 확인하면 추론이 GPU에서 수행되고 있는지 여부를 한 문장으로 바로 알 수 있다.


In [28]:
!ollama list

[GIN] 2025/09/15 - 14:24:49 | 200 |      33.159µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/15 - 14:24:49 | 200 |    2.118495ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED   
gemma3:27b     a418f5838eaf    17 GB    8 days ago    
gpt-oss:20b    aa4295ac10c3    13 GB    8 days ago    


In [32]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

# Ollama 연결
llm = ChatOllama(
    base_url="http://localhost:11434",  # ollama serve가 켜져 있는 주소
    model="gpt-oss:20b",
    temperature=0.3,
    num_ctx=2048,
)

# from langchain_community.llms import Ollama
# from langchain_core.messages import SystemMessage, HumanMessage

# llm = Ollama(
#     base_url="http://127.0.0.1:11434",  # ollama serve 실행 중인 주소
#     model="gpt-oss:20b",
#     temperature=0.3,
#     num_ctx=2048,
# )

In [33]:
# 단일 메시지
resp = llm.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)


[GIN] 2025/09/15 - 14:27:33 | 200 | 58.536564387s |       127.0.0.1 | POST     "/api/chat"
**장곡사(長谷寺) – 미륵불 괘불탱(미륵불 괘불탱)**  

---

### 1. 장곡사 개요  
- **위치**: 경기도 파주시 장곡면(서울특별시와 인접한 수도권 서부)  
- **설립**: 1943년 조선시대의 전통을 계승하며 개원한 사찰.  
- **특징**: ‘미륵불 괘불탱’이 대표적인 문화재로, 사찰 내부에 위치해 있는 미륵불을 수호하는 석탑이다.  

---

### 2. 미륵불 괘불탱이란?  
| 항목 | 내용 |
|------|------|
| **정의** | 미륵불(미래의 부처)을 수호·보호하기 위해 건축된 ‘석탑’ 형태의 불탱. 미륵불이 탑 내부에 안치되어 있는 구조. |
| **형태** | 1층(기반) → 2층(본체) → 3층(옥탑)으로 구성된 3층 석탑. 각 층은 돌을 깎아 만든 단단한 구조이며, 옥탑은 전통적인 삼각형 모양의 돌로 마감. |
| **크기** | 전체 높이 약 1.5m, 기초부는 0.4m, 본체는 0.8m, 옥탑은 0.3m. |
| **재료** | 지역에서 채취한 화강암을 사용해 석탑을 조각. |
| **미술적 특징** | 돌 표면에 연꽃, 구름, 용 등의 장식이 새겨져 있으며, 미륵불의 얼굴과 손짓이 섬세하게 조각되어 있다. |

---

### 3. 역사와 문화재 지정  
- **시대**: 고려시대(12세기) 초기에 건축된 것으로 추정.  
- **문화재 등재**: 1973년 국보 제134호로 지정.  
- **의의**: 고려시대 석탑 건축 기술과 미륵불 신앙이 결합된 대표적 예시로, 한국 불교 예술의 정수 중 하나로 평가된다.  

---

### 4. 미륵불(미래의 부처)과의 관계  
- **미륵불**: 불교에서 ‘미래의 부처’로, 인류에게 평화와 깨달음을 전할 예정인 부처.  
- **미륵불 괘불탱**: 미륵불을 안치해 두고, 사찰 방문객이 미륵불을 향

In [34]:
# 3) 테스트 import

# llm = ChatOllama(base_url="http://127.0.0.1:11434", model="gpt-oss:20b", temperature=0.3)
resp = llm.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 14:28:55 | 200 | 24.473311573s |       127.0.0.1 | POST     "/api/chat"
## 장곡사 미륵불 괘불탱이란?

### 1. 장곡사(장곡사) 개요  
- **위치**: 강원도 평창군 장곡면  
- **역사**: 748 년(대동왕 1년) 창건 → 1970년대 재건 → 현재는 ‘미륵불 괘불탱’이 가장 유명한 사찰  
- **특징**: 평창의 ‘산속 사찰’이라 불리며, 겨울철 눈꽃과 함께 아름다운 풍경을 자랑합니다.  

### 2. 미륵불 괘불탱이란?  
| 항목 | 내용 |
|------|------|
| **이름** | 미륵불 괘불탱 (Mireukbul Gwaebul-tang) |
| **형태** | 6.5 m 높이의 청동 조각, 머리에 ‘괘’(冠) 모양의 장식이 있는 ‘괘불’ 스타일 |
| **제작 시기** | 1975년(주조) → 1977년(설치) |
| **무게** | 약 30톤 |
| **재료** | 청동(구리-주석 합금) |
| **의의** | ‘미륵’은 미래에 세상을 구원할 미래 부처를 뜻하며, ‘괘불’은 부처가 입은 왕관(冠)을 상징해 권위와 신성함을 강조합니다. |

### 3. 제작 배경과 의미  
- **제작 배경**: 1970년대 한국은 경제 성장과 함께 문화재 보존·복원에 대한 관심이 높아졌습니다. 장곡사는 ‘미륵불’이라는 상징적 인물에 대한 재현을 통해 사찰의 신성함과 지역민의 희망을 표현하고자 청동 조각을 제작했습니다.  
- **의미**:  
  - **미륵**: 부처가 미래에 다시 세상에 나타나 평화와 번영을 가져올 것이라는 믿음.  
  - **괘(冠)**: 부처가 입는 왕관으로, 부처의 권위와 신성함을 상징.  
  - **청동**: 금속이 오래도록 견고하게 남아 신앙의 지속성을 상징.  

### 4. 문화·관광적 가치  
- **국가문화재**: 1978년 ‘국가문화재’로 지정되어 보호되고 있습니다.  
- **관광 명소**: 장곡사 자체가 ‘

### ollama - gemma3:27b

In [39]:
import os, subprocess

# 원하는 경로 지정 (여기선 ./models)
os.environ["OLLAMA_MODELS"] = "../../ollama"

# 디렉토리가 없으면 먼저 생성
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# gemma3:12b 모델 풀
subprocess.run(["ollama", "pull", "gemma3:27b"], check=True, env=os.environ)


[GIN] 2025/09/07 - 12:36:40 | 200 |      33.555µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/07 - 12:36:41 | 200 |  746.920851ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest ⠦ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gemma3'], returncode=0)

[GIN] 2025/09/07 - 12:36:52 | 200 |      26.533µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/07 - 12:36:52 | 200 |    6.479818ms |       127.0.0.1 | GET      "/api/tags"


In [35]:
# Ollama 연결
llm1 = ChatOllama(
    base_url="http://127.0.0.1:11434",  # ollama serve가 켜져 있는 주소
    model="gemma3:27b",
    temperature=0.3,
    num_ctx=2048,
)

In [36]:
# 단일 메시지
resp = llm1.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)

time=2025-09-15T14:29:52.985+09:00 level=INFO source=sched.go:540 msg="updated VRAM based on existing loaded models" gpu=GPU-9fc64c71-9800-370c-ee71-f6a0a869bc7a library=cuda total="39.5 GiB" available="26.3 GiB"
time=2025-09-15T14:29:53.580+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model ../../ollama/blobs/sha256-e796792eba26c4d3b04b0ac5adb01a453dd9ec2dfd83b6c59cbf6fe5f30b0f68 --port 43267"
time=2025-09-15T14:29:53.609+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-15T14:29:53.609+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:43267"
time=2025-09-15T14:29:54.496+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="843.7 GiB" free_swap="8.0 GiB"
time=2025-09-15T14:29:54.499+09:00 level=INFO source=memory.go:36 msg="new model will fit in available VRAM across minimum required GPUs, loading" model=../../ollama/blobs/sha256-e79679

[GIN] 2025/09/15 - 14:31:15 | 200 |         1m23s |       127.0.0.1 | POST     "/api/chat"
장곡사 미륵불 괘불탱은 경상남도 거창군 북상면 장곡사에 있는 괘불탱으로, 조선 후기에 제작된 불교 회화의 걸작입니다. 괘불탱은 불교 의식 때 법당에 걸어놓는 대형 불화로, 장곡사 괘불탱은 특히 뛰어난 예술성과 역사적 가치를 인정받아 국보 제135호로 지정되었습니다.

**주요 특징:**

*   **미륵보살의 광대한 세계:** 괘불탱의 중심에는 미래에 인간으로 다시 태어나 중생을 구제할 미륵보살이 웅장하게 자리하고 있습니다. 미륵보살은 화려한 보관을 쓰고, 다양한 장신구를 착용하고 있으며, 손에는 중생을 구제하는 상징인 여원인을 들고 있습니다. 미륵보살 주변에는 보살, 천인, 동자 등 다양한 부속 존재들이 묘사되어 있어 미륵보살의 광대한 세계를 표현하고 있습니다.
*   **화려하고 섬세한 표현:** 괘불탱은 전체적으로 화려하고 섬세한 표현이 돋보입니다. 채색 기법이 뛰어나고, 인물들의 표정과 옷 주름, 장신구 등이 정교하게 묘사되어 있습니다. 특히, 미륵보살의 얼굴은 자비롭고 온화한 인상을 주며, 보는 이로 하여금 경건한 마음을 갖게 합니다.
*   **조선 후기 불화의 특징:** 장곡사 괘불탱은 조선 후기 불화의 특징을 잘 보여줍니다. 이전 시대에 비해 인물 표현이 사실적으로 변화하고, 배경 묘사가 간략해지는 경향을 보입니다. 또한, 민화의 영향으로 인해 그림에 생동감이 더해졌습니다.
*   **제작 배경:** 장곡사 괘불탱은 18세기 후반에 제작된 것으로 추정됩니다. 정확한 제작 시기와 화가는 알려져 있지 않지만, 당시 불교계의 상황과 불화의 특징을 고려할 때, 조선 후기 대표적인 불화 화가 중 한 명이 제작했을 것으로 추정됩니다.

**장곡사 괘불탱의 가치:**

*   **예술적 가치:** 장곡사 괘불탱은 뛰어난 예술성을 바탕으로 불교 회화의 정수를 보여줍니다. 화려하고 섬세한 표현, 균형 

In [37]:
resp = llm1.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 14:32:10 | 200 | 22.733723709s |       127.0.0.1 | POST     "/api/chat"
안녕하세요! AI 문화유산 스마트 도슨트입니다. 장곡사 미륵불 괘불탱에 대해 자세히 설명해 드리겠습니다.

**장곡사 미륵불 괘불탱 (長谷寺彌勒佛掛佛幀)**

*   **지정:** 보물 제1357호
*   **소장:** 전라북도 정읍시 장곡사
*   **제작시기:** 조선 후기 (추정)
*   **크기:** 가로 6.8m, 세로 10.3m

**괘불탱이란?**

괘불탱은 절에서 중요한 의식을 치를 때 법당에 걸어놓는 대형 불교 회화입니다. 괘불탱은 단순히 그림이 아니라, 의식의 중요한 도구이자 신앙의 대상입니다. 특히 장곡사 괘불탱은 현존하는 괘불탱 중 가장 큰 규모를 자랑하며, 뛰어난 예술적 가치를 인정받고 있습니다.

**장곡사 괘불탱의 특징**

*   **미륵보살의 광대한 세계:** 괘불탱의 중앙에는 미래에 인간으로 다시 태어나 중생을 구제할 미륵보살이 웅장하게 자리하고 있습니다. 미륵보살은 화려한 보관을 쓰고, 온화한 미소를 짓고 있으며, 손에는 중생을 구제하는 상징인 여의주를 들고 있습니다. 미륵보살 주변에는 다양한 보살과 천인, 그리고 극락 세계를 상징하는 연꽃과 구름이 묘사되어 있어, 광대한 미륵보살의 세계를 표현하고 있습니다.
*   **화려하고 섬세한 채색:** 괘불탱은 밝고 화려한 오채를 사용하여 그려졌습니다. 특히, 미륵보살의 옷과 보관, 그리고 주변 보살들의 장신구는 금박을 사용하여 빛을 발하며, 섬세하고 정교한 묘사로 보는 이의 감탄을 자아냅니다.
*   **조선 후기 불화의 정수:** 장곡사 괘불탱은 조선 후기 불화의 특징을 잘 보여줍니다. 화려하고 장식적인 표현, 그리고 섬세하고 정교한 묘사는 당시 불화의 높은 수준을 보여줍니다. 또한, 괘불탱에는 당시 사회의 문화와 사상을 반영하는 다양한 상징과 문양이 담겨 있어, 역사적, 문화적 가치가 높습니다.
*   **희귀한 구도